# Telemática

En este cuaderno se presenta una guía de configuración de algunos servicios en máquinas Ubuntu en la plataforma AWS.

Realizado por:

Ferney O. Amaya Fernández <BR>
ferney.amaya@gmail.com

#1.Regiones, Zonas de Disponibilidad y VPCs

Amazon cuente con regiones en todo el mundo, por ejemplo:
- us-east-1 (Norte de Virginia, EE. UU.) con 6 zonas de disponibilidad
- eu-west-1 (Irlanda)
- ap-southeast-1 (Singapur)

En cada región se tienen varias Zonas de Disponibilidad (*Availability Zones, AZ*). Una AZ es un grupo de centros de datos con alimentación, refrigeración y seguridad física independientes.

[Mayor información aquí.](https://aws.amazon.com/es/about-aws/global-infrastructure/)

En una VPC (*Virtual Private Cloud*) pueden lanzarse varios recursos de AWS en una red virtual lógica aislada que se asemeja a una red tradicional que conforma un centro de datos.

[Mayor información aquí.](https://docs.aws.amazon.com/es_es/vpc/latest/userguide/what-is-amazon-vpc.html)

#2 Paso a paso para crear una VPC manual

La VPC y sus componentes pueden crearse en forma automática y en forma manual. Aquí se presenta una breve guía para crearlas en forma manual.

Se creará una VPC con 1 subred pública y 1 subred privada. Se empleará el bloque CIDR IPv4 10.2.0.0/16 para crear subredes 10.2.0.0/18.

##1.1 **Paso 1:** Crear la VPC

La configuración a seleccionar es la siguiente:
- Resources to create: VPC only
- IPv4 CIDR manual input:
- IPv6 CIDR block: Amazon-provided IPv6 CIDR block
- Tenancy: Default (sin HW dedicado, más barato)

##1.2 **Paso 2:** Crear las subredes

Las subredes públicas y privadas se crean igual. La diferencia estará más adelante en la tabla de enrutamiento al conectar o no con la puerta de enlace.

- Seleccione la VPC
- Identificar y planear el direccionamiento IPv4 e IPv6
- Asigne un nombre que indique si será pública o privada
- Seleccione AZ
- Seleccione IPv4 e IPv6 subnet CIDR block

##1.3 **Paso 3:** Crear el GW (Internet gateways)

El GW (*Gateway*) o puerta de enlace:
- Crear el GW
- Editarlo para conectarlo con la VPC:
 * En Actions selecionar Atach to VPC
 * Seleccionar la VPC

##1.4 **Paso 4:** Crear las Tablas de Enrutamiento

Es suficiente una tabla para las subredes públicas con el mismo GW y otra para las privadas.
- Seleccione la VPC y cree la tabla de enrutamiento
- Edite la tabla de enrutamiento para asociarla a la subred:
  - Seleccione la pestaña Subnet Associations
  - En la sección Subnet without explicit associations editar para selecionar la subred



##1.5 **Paso 5:** Ajustar las rutas en las Tablas de Enrutamiento

La tabla de enrutamiento para la subred pública debe ajustarse para que quede de la siguiente forma (el siguiente es un ejemplo, los datos deben ajustarse dependiendo de las direcciones IP):

|Destino|Destino|
|--|--|
|::/0|igw-0d7ec8f0c00a668ba|
|2600:1f10:4d0a:5700::/56|local|
|0.0.0.0/0|igw-0d7ec8f0c00a668ba|
|10.2.0.0/16|local|






#2.Enlace SSH entre cliente y servidor

##2.1 Revisión e instalación

Verificar cliente:
```
ssh –V
```

Verificar servidor:
```
sudo systemctl status ssh
```

##2.2 Usuario en el servidor

Verificar en el servidor que el usuario existe. A modo de prueba, se sugiere crear el usuario *tele* con contraseña *tele*.

##2.3 En el servidor ajustar la configuración

En el archivo de configuración del servidor:

```
/etc/ssh/sshd_config
```

Quitarle el comentario a las siguientes opciones:
- Port 22
- PasswordAuthentication yes
- PubkeyAuthentication yes


##2.4 En el cliente generar y verificar el par de llaves

Para generar el par de llaves se sugiere:
```
ssh-keygen -t rsa -b 4096
```
Las llaves se crean en las siguientes ubicaciones:
- ~/.ssh/id_rsa	: Clave privada, no compartir.
- ~/.ssh/id_rsa.pub	: Clave pública, puede compartirse
Observe las llaves.

##2.5 Desde el cliente copiar la clave pública al servidor

El comando para enviar desde el cliente la clave pública al servidor es el siguiente:
```
 ssh-copy-id username@IP_servidor
```

Si el comando anterior no funciona, debe copiarse la clave manualmente. Siga los siguientes pasos:

- Paso 1: El el Servidor, ingrese a la sesión del usuario
- Paso 2: Varificar en la carpeta de home del usuario, que la carpeta .ssh existe. Si no existe, debe crearla y debe quedar con los siguientes privilegios:
```
 drwx------      .ssh
```
- Paso 3: Verifique dentro de la carpeta .ssh que el archivo authorized_keys existe. Si no existe, debe crearlo y debe quedar con los siguientes privilegios:
```
 -rw-------      authorized_keys
```
- Paso 4: Copie manualmente la llave pública creada en el Cliente, al archivo *authorized_keys* en el servidor.




#3.UFW
La configuración del servicio UFW (*Uncomplicated FireWall*) es la siguiente:

##3.1. Configurar las políticas por defecto
```
sudo ufw default deny incoming​
sudo ufw default allow outgoing​
```

##3.2. Habilitar servicios

El mapeo de servicios a números de puertos se encuentra en el archivo:
```
/etc/services
```
Para agregar un servicio puede hacerse por el nombre de servicio o por número de puerto:
```
sudo ufw allow servicio​
sudo ufw allow numero_puerto​
```

##3.3. Controlar UFW​
Para controlar el servicio UFW se emplean los siguientes comandos:
```
sudo ufw enable​
sudo ufw status verbose​
sudo ufw disable​
sudo ufw status numbered
```